In [ ]:
import numpy as np

In [ ]:
import os

root = '/data/xp/test_only_for_test'
identification_root = os.path.join(root, 'identification/test')
quantification_root = os.path.join(root, 'quantification/test')
data_name = 'ttp_20230702_CCS'

target = np.load(os.path.join(identification_root, 'target', data_name, 'collection.npy'), allow_pickle=True).item()
target['matchedMs2'] = target['matchedMs2'] / np.max(target['matchedMs2'])

decoy = np.load(os.path.join(identification_root, 'decoy', data_name, 'collection.npy'), allow_pickle=True).item()
decoy['matchedMs2'] = decoy['matchedMs2'] / np.max(decoy['matchedMs2'])

quant = np.load(os.path.join(quantification_root, data_name, 'collection.npy'), allow_pickle=True).item()

In [ ]:
from dia_dl.generatedata.utils.utils import read_json
from dia_dl.generatedata.fullDataInMemory.genDataLoader import testDataLoader

batch_size = 4096
model_configs = read_json('./configs/model_configs.json')
identify_configs = model_configs['identification']
quant_configs = model_configs['quantification']

target = testDataLoader(target, identify_configs['features']['read'], batch_size)
decoy = testDataLoader(decoy, identify_configs['features']['read'], batch_size)
quant = testDataLoader(quant, quant_configs['features']['read'], batch_size)

In [ ]:
identify_configs['features']['read']

In [27]:
import torch

from dia_dl.model.model import identification_model, quantification_model 
from dia_dl.model.model_utils import classfier_test, regression_test

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

identify_model = identification_model(6, 6).to(device)
quant_model = quantification_model(6, 6).to(device)

identify_modelstate = torch.load('./model/identification.ckpt')
quant_modelstate = torch.load('./model/quantification.ckpt')

identify_model.load_state_dict(identify_modelstate)
quant_model.load_state_dict(quant_modelstate)

target_result = classfier_test(
    identify_model, 
    device,
    target,
    identify_configs
)

decoy_result = classfier_test(
    identify_model, 
    device,
    decoy,
    identify_configs
)

quant_result = regression_test(
    quant_model, 
    device,
    quant,
    quant_configs
)

In [28]:
quantity, info = quant_result['quantity'], quant_result['info']
indices = (quantity > 0).nonzero()
quantity = quantity[indices]
info_sequence = info[indices]

files = set(info_sequence[:, 0])

quant = {
    f: {}
    for f in files
}

In [29]:
quantity, info_sequence

(array([ 292.23096,  341.27765,  289.60272, ...,  298.55612,  293.87073,
        1336.8391 ], dtype=float32),
 array([['ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-3_Slot1-10_1_12903',
         ('_C[Carbamidomethyl (C)]LC[Carbamidomethyl (C)]PEGFSLSSSGR_', 2)],
        ['ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-3_Slot1-10_1_12903',
         ('_LGQSLDC[Carbamidomethyl (C)]NAEVYVVPWEKK_', 3)],
        ['ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-3_Slot1-10_1_12903',
         ('_MDC[Carbamidomethyl (C)]QEC[Carbamidomethyl (C)]PEGYR_', 2)],
        ...,
        ['ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-1_Slot1-10_1_12901',
         ('_AFSFSLPRPGR_', 2)],
        ['ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-1_Slot1-10_1_12901',
         ('_DSFHLDEQFTVPVEM[Oxidation (M)]MQAR_', 3)],
        ['ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-1_Slot1-10_1_12901',
         ('_GGETSEMYLIQPDSSVKPYR_', 3)]], dtype=object))

In [30]:
file, modified_charge = info[0]
file, modified_charge

('ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-3_Slot1-10_1_12903',
 ('_C[Carbamidomethyl (C)]LC[Carbamidomethyl (C)]PEGFSLSSSGR_', 2))

In [31]:
import pandas as pd
from dia_dl.model.utils import generate_modifiedPeptide_to_strippedPeptide_map, get_dl_identify_modified

raw_library_path = '/data/xp/raw_library/AD8-300S-directDIA.tsv'
raw_library = pd.read_csv(raw_library_path, sep='\t', low_memory=False)

modified_to_stripped_map = generate_modifiedPeptide_to_strippedPeptide_map(
    library=raw_library
)

identify_result = get_dl_identify_modified(
    t = target_result,
    d = decoy_result
)

for q, info in zip(quantity, info_sequence):
    file, modified_charge = info
    if modified_charge in identify_result[file]:
        quant[file][modified_charge] = {
            'quantity': q,
            'strippedSequence': modified_to_stripped_map[modified_charge[0]]
        }

ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-3_Slot1-10_1_12903


 19%|█▉        | 2693/13926 [00:00<00:00, 1115801.71it/s]


threshold=0.048567034304142, modified_peptide_length=11233
ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-2_Slot1-10_1_12902


 19%|█▉        | 2670/13952 [00:00<00:00, 1138435.67it/s]


threshold=0.046587806195020676, modified_peptide_length=11282
ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-5_Slot1-10_1_12905


...

threshold=0.05498975142836571, modified_peptide_length=11135
ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-4_Slot1-10_1_12904


 19%|█▉        | 2664/14035 [00:00<00:00, 1122977.47it/s]


threshold=0.047586046159267426, modified_peptide_length=11371
ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-1_Slot1-10_1_12901


 19%|█▉        | 2669/13847 [00:00<00:00, 956394.48it/s]

threshold=0.048847850412130356, modified_peptide_length=11178


In [32]:
quant

{'ttp_20230702_CCS_Plasma_trypsin_Mix1_20min_system-test-4_Slot1-10_1_12904': {('_MKQTVEAM[Oxidation (M)]K_',
   2): {'quantity': 279.56992, 'strippedSequence': 'MKQTVEAMK'},
  ('_C[Carbamidomethyl (C)]LC[Carbamidomethyl (C)]PEGFSLSSSGR_',
   2): {'quantity': 281.53897, 'strippedSequence': 'CLCPEGFSLSSSGR'},
  ('_LGQSLDC[Carbamidomethyl (C)]NAEVYVVPWEKK_', 3): {'quantity': 291.808,
   'strippedSequence': 'LGQSLDCNAEVYVVPWEKK'},
  ('_LESEETMVLEAHDAQGDVPVTVTVHDFPGKK_', 5): {'quantity': 348.301,
   'strippedSequence': 'LESEETMVLEAHDAQGDVPVTVTVHDFPGKK'},
  ('_QVLDNLTMEK_', 2): {'quantity': 453.4893,
   'strippedSequence': 'QVLDNLTMEK'},
  ('_YGQPLPGYTTK_', 2): {'quantity': 451.28085,
   'strippedSequence': 'YGQPLPGYTTK'},
  ('_KSLLATMK_', 2): {'quantity': 318.0626, 'strippedSequence': 'KSLLATMK'},
  ('_KPSLSVQPGPVVAPGEK_', 3): {'quantity': 495.69287,
   'strippedSequence': 'KPSLSVQPGPVVAPGEK'},
  ('_LPNTLKPDSYR_', 2): {'quantity': 306.5416,
   'strippedSequence': 'LPNTLKPDSYR'},
  ('_TYC[C

In [33]:
import pandas as pd

df_sequence = []

for file, pep_quant_dict in quant.items():
    quantity_col = f'{file}.Quantity'
    df = pd.DataFrame(columns=['modifiedPeptide', 'charge', 'strippedSequence', quantity_col])
    for modified_charge, info in pep_quant_dict.items():
        # df 插入一条数据, 需要使用定性的结果进行筛选
        # 1. 筛选出定性到的肽段
        # 2. 将同一个的 stripped_sequence 肽段的定量结果进行合并
        df.loc[len(df)] = [
            modified_charge[0],
            modified_charge[1],
            info['strippedSequence'],
            info['quantity']
        ]
    groups = df.groupby('strippedSequence')

    for k, v in groups.groups.items():
        group = groups.get_group(k)
        df.loc[v, quantity_col] = group[quantity_col].sum()
    df_sequence.append(df)

In [34]:
from typing import Sequence
from functools import reduce

def merge(df1, df2, on_columns: Sequence[str]):
    df = pd.merge(df1, df2, how='outer', on=on_columns)
    return df

on_columns = ['modifiedPeptide', 'charge', 'strippedSequence']
df_generator = (df for df in df_sequence)
df = reduce(lambda x, y: merge(x, y, on_columns), df_generator)
file_name_map = {
    f'{file}.Quantity': f'[{i + 1}] {file}.Quantity'
    for i, file in enumerate(quant.keys())
}

df.rename(columns=file_name_map, inplace=True)

df.to_csv('r.tsv', index=False, sep='\t')

In [35]:
from typing import Sequence
import pandas as pd

def cv_filter(x: pd.Series, cv: float):
    values = x.values[1:]
    return (values.std() / values.mean()) < cv

def get_sample_quant_fianlly(x: pd.Series, sample_columns: Sequence[str], is_A: bool):
    if is_A:
        return pd.Series([x[0], x[sample_columns].mean()], index=['strippedSequence', 'A'])
    else:
        return pd.Series([x[0], x[sample_columns].mean()], index=['strippedSequence', 'B'])

def get_quant_view_dataframe(df: pd.DataFrame, cv: float):
    """
        选出在所有 QC 样本均定量到的肽段, cv < 0.1/0.2

        Parameters:
        ---
        -   df: 定量得到的 dataframe
        -   cv: 筛选的阈值选择

        Returns:
        ---
        -   df_view: 筛选后的 dataframe
    """
    qc_file_nums = len(df.columns) - 1
    # print(qc_file_nums)
    start = 1
    # 索引左闭右开
    A_index_start, A_index_end = start, qc_file_nums // 2 + start
    B_index_start = (qc_file_nums + 1) // 2 + start
    A_samples, B_samples = df.columns[A_index_start:A_index_end].values, df.columns[B_index_start:].values
    # print(A_samples, B_samples)

    df_view = df.dropna(axis=0, how='any').reset_index(drop=True)
    # print(df_view.columns)

    df_view_group = df_view.groupby('strippedSequence')
    # 取 first 值
    view_quant_df = df_view_group.agg('first').reset_index(drop=False)
    cv_before_nums = len(view_quant_df)
    # print(cv_before_nums)
    cv_df = view_quant_df[view_quant_df.apply(cv_filter, axis=1, args=(cv, ))]
    cv_df = cv_df.reset_index(drop=True)
    A = cv_df.apply(get_sample_quant_fianlly, args=(A_samples, True, ), axis=1)
    B = cv_df.apply(get_sample_quant_fianlly, args=(B_samples, False, ), axis=1)
    # print(A.columns, B.columns)
    sample_df = pd.merge(left=A, right=B, how='inner')
    cv_after_nums = len(cv_df)
    return sample_df, cv_before_nums, cv_after_nums

In [38]:
def process(df):
    df['A/B'] = np.log2(df['A'] / df['B'])
    df['B'] = np.log2(df['B'])

cv = 0.1

df = pd.read_csv('r.tsv', sep='\t')
df = df[df.columns[2:]]
df, _, _ = get_quant_view_dataframe(df, cv)
process(df)

In [39]:
df

,strippedSequence,A,B,A/B
0,AAAAAAALQAK,258.225725,8.161770,-0.149281
1,AAALAHLDR,296.337470,8.249930,-0.038833
2,AAATLMSER,321.507170,8.409163,-0.080456
3,AACLPLPGYR,332.342820,8.141425,0.235103
4,AADAEAEVASLNR,293.178390,8.390699,-0.195064
...,...,...,...,...
2828,YYIAASYVK,333.665685,8.288350,0.093910
2829,YYPYQSFQTPQHPSFLFQDKR,312.679600,8.222562,0.065980
2830,YYTSASGDEMVSLK,776.585550,9.519974,0.081027
2831,YYWGGQYTWDMAK,24342.591000,14.661544,-0.090349


In [ ]:
target_scores = target_result['score']
decoy_scores = decoy_result['score']

In [ ]:
target_scores.mean(), decoy_scores.mean(), target_scores.max(), target_scores.min(), decoy_scores.max(), decoy_scores.min()

In [ ]:
quant_result['quantity'] = quant_result['quantity'][quant_result['quantity'] > 0]
quant_result['quantity'] = np.log2(quant_result['quantity'])
quant_result['quantity'].max(), quant_result['quantity'].min(), quant_result['quantity'].mean()

In [ ]:
import pandas as pd
import seaborn as sns

sns.set_palette('Set2')

hue = ['target'] * len(target_scores)
hue.extend(['decoy'] * len(decoy_scores))

score = np.concatenate((target_scores, decoy_scores), axis=0)

df = pd.DataFrame(data={
    'scores': score,
    'hue': hue
})

sns.histplot(data=df, x='scores', hue='hue', bins=50)